# [SC57 - Working with big, multi-dimensional geoscientific datasets in Python: a tutorial introduction to xarray](http://meetingorganizer.copernicus.org/EGU2017/session/25651)  
  
  
Original notebook by [Stephan Hoyer](http://stephanhoyer.com), Rossbypalooza, 2016.  
Modified by Edward Byers, Matthew Gidden and [Fabien Maussion](http://fabienmaussion.info/) for EGU General Assembly 2017, Vienna, Austria
Modified by C. Gentemann for GHRSST Science Team Tutorial 2019, Rome, Italy
Modified to include podaacpy by C.Gentemann with help from @lewismc for OceanObs19
  
**Convenors**
* [Dr Chelle Gentemann](mailto:gentemann@esr.org)    - Earth and Space Research, USA
* [Dr Marisol Garcia-Reyes](mailto:marisolgr@faralloninstitute.org)  - Farallon Institute, USA 
-------------

# Structure of this tutorial

1. Opening data
1. Collocating satellite data with a cruise dataset



# 1. Key features of `xarray`

-------------------

## Import python packages

You are going to want numpy, pandas, matplotlib.pyplot and xarray

In [1]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

#for search capabilites import podaacpy
import podaac.podaac as podaac
import podaac.podaac_utils as putil
# then create an instance of the Podaac class
p = podaac.Podaac()

## A nice cartopy tutorial is [here](http://earthpy.org/tag/visualization.html)

# Collocate a Saildrone cruise with PODAAC MUR SST data

### The Saildrone cruise is 2 months long.  The MUR SST data is daily 1 km global SSTs

`xarray`can open multiple files at once using string pattern matching.  
  
  In this case we open all the files that match our `filestr`, i.e. all the files for the 2080s. 
  
  Each of these files (compressed) is approximately 800 MB.

# Collocating Saildrone cruise data with MUR SSTs 

* read in the Saildrone data 

In [3]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds_usv = xr.open_dataset(url)
ds_usv

<xarray.Dataset>
Dimensions:               (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory            (trajectory) float32 1002.0
    time                  (trajectory, obs) datetime64[ns] ...
    latitude              (trajectory, obs) float64 ...
    longitude             (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                   (trajectory, obs) float64 ...
    COG                   (trajectory, obs) float64 ...
    HDG                   (trajectory, obs) float64 ...
    HDG_WING              (trajectory, obs) float64 ...
    ROLL                  (trajectory, obs) float64 ...
    PITCH                 (trajectory, obs) float64 ...
    WING_ANGLE            (trajectory, obs) float64 ...
    BARO_PRES_MEAN        (trajectory, obs) float64 ...
    BARO_PRES_STDDEV      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN         (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV       (trajectory, obs) float64 ...
    RH_MEAN        

Input in the start and end time manually

In [6]:
str_start_time = '2018-04-12T02'
str_end_time = '2018-06-10T18'

start:  2018-04-11T18:04:00Z end:  2018-06-11T05:06:00Z


In [8]:
ds_usv2 = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'longitude':'lon','latitude':'lat'})
ds_usv_subset = ds_usv2.sel(time=slice(str_start_time,str_end_time)) 
start_time=pd.to_datetime(str(ds_usv2.time.min().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 
end_time=pd.to_datetime(str(ds_usv2.time.max().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 
ds_usv_subset
print('start: ',start_time,'end: ',end_time)

start:  2018-04-11T18:04:00Z end:  2018-06-11T05:06:00Z


Collocate with Ocean Color Observations

In [11]:
## YYYY-MM-DD
start_date = np.datetime64(str_start_time)
end_date   = np.datetime64(str_end_time)
num_days = (end_date - start_date).tolist().days

# variable to load
VAR = 'CHL'

# algorithm
ALG = 'chl_ocx'

# Binning period
BIN = 'DAY'  # DAY, 8D, MO, R32

# Spatial resolution
SRES = '9km'   # 4km, 9km

dap_urls = []

url_base = "https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/"

In [12]:
for ii in range(num_days):
    curr_date = start_date + ii
    
    curr_year = curr_date.tolist().year
    ref_date = np.datetime64('{0:d}-01-01'.format(curr_year))
    
    day_num = 1 + (curr_date - ref_date).tolist().days
    
    file_url = url_base + \
                '{0:d}/{1:03d}/A{0:d}{1:03d}'.format(curr_year, day_num) + \
                '.L3m_{0}_{1}_{2}_{3}'.format(BIN, VAR, ALG, SRES) + \
                '.nc'
    
    dap_urls += [file_url]
    ds = xr.open_dataset(file_url)

In [13]:
ds

<xarray.Dataset>
Dimensions:  (eightbitcolor: 256, lat: 2160, lon: 4320, rgb: 3)
Coordinates:
  * lat      (lat) float32 89.958336 89.875 89.79167 ... -89.87501 -89.958336
  * lon      (lon) float32 -179.95833 -179.875 ... 179.87502 179.95836
Dimensions without coordinates: eightbitcolor, rgb
Data variables:
    palette  (rgb, eightbitcolor) int8 ...
    chl_ocx  (lat, lon) float32 ...
Attributes:
    _NCProperties:                     version=1|netcdflibversion=4.4.1.1|hdf...
    product_name:                      A2018104.L3m_DAY_CHL_chl_ocx_9km.nc
    instrument:                        MODIS
    title:                             MODISA Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          Aqua
    temporal_range:                    day
    processing_version:                2018.0
    date_created:                      2018-06-05T21:06:46.000Z
    history:                           l3map

In [9]:
#dataset_id = 'PODAAC-GHGMR-4FJ04'  #MUR SST looked up on podaac website
dataset_id = 'PODAAC-GHK10-41N01'  #smaller data
gresult = p.granule_search(dataset_id=dataset_id,
                           start_time=start_time,
                           end_time=end_time,
                           items_per_page='100')
urls = putil.PodaacUtils.mine_opendap_urls_from_granule_search(gresult)
urls = [w[:-5] for w in urls]  #remove html from urls

In [5]:
ds_sst = xr.open_mfdataset(urls,coords='minimal')
ds_sst

<xarray.Dataset>
Dimensions:         (lat: 1801, lon: 3600, time: 62)
Coordinates:
  * lat             (lat) float32 90.0 89.9 89.8 89.700005 ... -89.8 -89.9 -90.0
  * lon             (lon) float32 -180.0 -179.90001 -179.8 ... 179.8 179.90001
  * time            (time) object 2018-04-12 00:00:00 ... 2018-06-12 00:00:00
Data variables:
    analysed_sst    (time, lat, lon) float32 dask.array<shape=(62, 1801, 3600), chunksize=(1, 1801, 3600)>
    analysis_error  (time, lat, lon) float32 dask.array<shape=(62, 1801, 3600), chunksize=(1, 1801, 3600)>
    mask            (time, lat, lon) int16 dask.array<shape=(62, 1801, 3600), chunksize=(1, 1801, 3600)>
Attributes:
    Conventions:            NAVO_netcdf_v3.4
    title:                  NAVOCEANO 10km Analyzed Sea Surface Temperature
    DSD_entry_id:           NAVO-L4HR1m-GLOB-K10_SST
    reference:              https://oceanography.navy.mil/legacy/web
    institution:            Naval Oceanographic Office
    contact:                doug.m

How big is all this data uncompressed? Will it fit into memory?
Use `.nbytes` / 1e9  to convert it into gigabytes

In [6]:
ds_sst.nbytes / 1e9  

4.0198541

## The NCEI trajectory format uses 'obs' as the coordinate.  This is an example of an 'older' style of data formatting that doesn't really mesh well with modern software capabilities. 

* So, let's change that by using [.swap_dims](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.swap_dims.html) to change the coordinate from `obs` to `time`
* Another thing, `latitude` and `longitude` are just long and annoying, lets [.rename](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.rename.html) them to `lat` and `lon`

* Finally, the first and last part of the cruise the USV is being towed, so let's only include data from `2018-04-12T02` to `2018-06-10T18`


# Xarray interpolation won't run on chunked dimensions.  
1. First let's subset the data to make it smaller to deal with by using the cruise lat/lons
    * Find the max/min of the lat/lon using `.lon.min().data`

1. Now load the data into memory (de-Dask-ify) it using `.load()`  


In [7]:
#Step 1 from above
print('min max lat lon:', ds_usv_subset.lon.min().data,ds_usv_subset.lon.max().data,ds_usv_subset.lat.min().data,ds_usv_subset.lat.max().data)
lon_min,lon_max = ds_usv_subset.lon.min().data,ds_usv_subset.lon.max().data
lat_min,lat_max = ds_usv_subset.lat.min().data,ds_usv_subset.lat.max().data
subset = ds_sst.sel(lon=slice(lon_min,lon_max),
                    lat=slice(lat_min,lat_max))

#Step 2 from above
subset.load()

min max lat lon: -125.55297279999999 -115.5226624 28.0176832 37.6797408


<xarray.Dataset>
Dimensions:         (lat: 0, lon: 100, time: 62)
Coordinates:
  * lat             (lat) float32 
  * lon             (lon) float32 -125.5 -125.4 -125.3 ... -115.700005 -115.6
  * time            (time) object 2018-04-12 00:00:00 ... 2018-06-12 00:00:00
Data variables:
    analysed_sst    (time, lat, lon) float32 
    analysis_error  (time, lat, lon) float32 
    mask            (time, lat, lon) int16 
Attributes:
    Conventions:            NAVO_netcdf_v3.4
    title:                  NAVOCEANO 10km Analyzed Sea Surface Temperature
    DSD_entry_id:           NAVO-L4HR1m-GLOB-K10_SST
    reference:              https://oceanography.navy.mil/legacy/web
    institution:            Naval Oceanographic Office
    contact:                doug.may@navy.mil
    GDS_version_id:         v1.0-rev1.7
    netcdf_version_id:      netcdf v3.4
    creation_date:          created: 2018-04-12 07:00:03
    history:                Created from k10.out, k10a.out
    operational_status:   

# Collocate USV data with MUR data
There are different options when you interpolate.  First, let's just do a linear interpolation using [.interp()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.interp.html#xarray.Dataset.interp)

`Dataset.interp(coords=None, method='linear', assume_sorted=False, kwargs={}, **coords_kwargs))`



In [8]:
ds_collocated = subset.interp(lat=ds_usv_subset.lat,lon=ds_usv_subset.lon,time=ds_usv_subset.time,method='linear')

IndexError: index -1 is out of bounds for axis 0 with size 0

# Collocate USV data with MUR data
There are different options when you interpolate.  First, let's just do a nearest point rather than interpolate the data
`method = 'nearest'`

In [ ]:
ds_collocated_nearest = subset.interp(lat=ds_usv_subset.lat,lon=ds_usv_subset.lon,time=ds_usv_subset.time,method='nearest')

## Now, calculate the different in SSTs and print the [.mean()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray.DataArray.mean) and [.std()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.std.html#xarray.DataArray.std)
For the satellite data we need to use `sst` and for the USV data we need to use `TEMP_CTD_MEAN`

In [ ]:
dif = ds_collocated_nearest.sst-ds_usv.TEMP_CTD_MEAN
print('mean difference = ',dif.mean().data)
print('STD = ',dif.std().data)

# A larger STD that isn't reflective of uncertainty in the observation
The collocation above will result in multiple USV data points matched with a single satellite
observation.    The USV is sampling every 1 min and approximately few meters, while the satellite
is an average over a footprint that is interpolated onto a daily mean map.  While calculating the mean would results in a valid mean, the STD would be higher and consist of a component that reflects the uncertainty of the USV and the satellite and a component that reflects the natural variability in the region that is sampled by the USV

Below we use the 'nearest' collocation results to identify when multiple USV data are collcated to
a single satellite observation.
This code goes through the data and creates averages of the USV data that match the single CCMP collocated value.


In [ ]:
ilen,index = ds_collocated_nearest.dims['time'],0
ds_tem = ds_collocated_nearest.copy(deep=True)
duu, duv1, duv2, dlat, dlon, dut = [],[],[],[],[],np.empty((),dtype='datetime64')
while index <= ilen-2:
    index += 1
    if np.isnan(ds_collocated_nearest.analysed_sst[index]):
        continue
    if np.isnan(ds_tem.analysed_sst[index]):
        continue
   # print(index, ilen)
    iend = index + 1000
    if iend > ilen-1:
        iend = ilen-1
    ds_tem_subset = ds_tem.analysed_sst[index:iend]
    ds_usv_subset2sst = ds_usv_subset.TEMP_CTD_MEAN[index:iend]
    ds_usv_subset2uwnd = ds_usv_subset.UWND_MEAN[index:iend]
    ds_usv_subset2vwnd = ds_usv_subset.VWND_MEAN[index:iend]
    ds_usv_subset2lat = ds_usv_subset.lat[index:iend]
    ds_usv_subset2lon = ds_usv_subset.lon[index:iend]
    ds_usv_subset2time = ds_usv_subset.time[index:iend]
    cond = ((ds_tem_subset==ds_collocated_nearest.analysed_sst[index]))
    notcond = np.logical_not(cond)
    #cond = ((ds_tem.analysed_sst==ds_collocated_nearest.analysed_sst[index]))
    #notcond = np.logical_not(cond)
    masked = ds_tem_subset.where(cond)
    if cond.sum().data==0:  #don't do if data not found
        continue
    if cond.sum().data>800:
        print(cond.sum().data,index,ds_tem.time[index].data)
    masked_usvsst = ds_usv_subset2sst.where(cond,drop=True)
    masked_usvuwnd = ds_usv_subset2uwnd.where(cond,drop=True)
    masked_usvvwnd = ds_usv_subset2vwnd.where(cond,drop=True)
    masked_usvlat = ds_usv_subset2lat.where(cond,drop=True)
    masked_usvlon = ds_usv_subset2lon.where(cond,drop=True)
    masked_usvtime = ds_usv_subset2time.where(cond,drop=True)
    duu=np.append(duu,masked_usvsst.mean().data)
    duv1=np.append(duv1,masked_usvuwnd.mean().data)
    duv2=np.append(duv2,masked_usvvwnd.mean().data)
    dlat=np.append(dlat,masked_usvlat.mean().data)
    dlon=np.append(dlon,masked_usvlon.mean().data)
    tdif = masked_usvtime[-1].data-masked_usvtime[0].data
    mtime=masked_usvtime[0].data+np.timedelta64(tdif/2,'ns')
    if mtime>dut.max():
        print(index,masked_usvtime[0].data,(masked_usvtime[-1].data-masked_usvtime[0].data)/1e9)
    dut=np.append(dut,mtime)
    ds_tem.analysed_sst[index:iend]=ds_tem.analysed_sst.where(notcond)
#    ds_tem=ds_tem.where(notcond,np.nan)  #masked used values by setting to nan
dut2 = dut[1:]  #remove first data point which is a repeat from what array defined    
ds_new=xr.Dataset(data_vars={'sst_usv': ('time',duu),'uwnd_usv': ('time',duv1),'vwnd_usv': ('time',duv2),
                             'lon': ('time',dlon),
                             'lat': ('time',dlat)},
                  coords={'time':dut2})
#ds_new.to_netcdf('F:/data/cruise_data/saildrone/baja-2018/mur_downsampled_usv_data2.nc')

In [ ]:
dif = ds_collocated_nearest.sst-ds_usv.TEMP_CTD_MEAN
print('mean difference = ',dif.mean().data)
print('STD = ',dif.std().data)
